In [1]:
import geopandas as gpd
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn.cluster


In [2]:
import numpy as np

In [3]:
import folium

In [4]:
import pandas as pd


In [6]:
# geojson 파일 불러옴
파출소지구대 = gpd.read_file('data/suwon_police.geojson')
파출소지구대

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,POL_NM,geometry
0,41111129,Pajang-dong,파장동,노송지구대,"MULTIPOLYGON (((127.01079 37.31908, 127.01047 ..."
1,41111130,Jeongja-dong,정자동,노송지구대,"MULTIPOLYGON (((126.98697 37.31147, 126.98705 ..."
2,41111131,Imok-dong,이목동,노송지구대,"MULTIPOLYGON (((126.98105 37.32685, 126.98160 ..."
3,41111132,Yuljeon-dong,율전동,율천파출소,"MULTIPOLYGON (((126.97444 37.30857, 126.97466 ..."
4,41111133,Cheoncheon-dong,천천동,노송지구대,"MULTIPOLYGON (((126.98169 37.30624, 126.98170 ..."
5,41111134,Yeonghwa-dong,영화동,화서문지구대,"MULTIPOLYGON (((127.02641 37.29733, 127.02597 ..."
6,41111135,Songjuk-dong,송죽동,장안문지구대,"MULTIPOLYGON (((127.01135 37.31892, 127.01146 ..."
7,41111136,Jowon-dong,조원동,장안문지구대,"MULTIPOLYGON (((127.02290 37.30862, 127.02336 ..."
8,41111137,Yeonmu-dong,연무동,창룡문지구대,"MULTIPOLYGON (((127.03274 37.30082, 127.03258 ..."
9,41111138,Sanggwanggyo-dong,상광교동,장안문지구대,"MULTIPOLYGON (((127.03171 37.33646, 127.03162 ..."


In [7]:
# 격자 데이터
data_gid = 파출소지구대[['POL_NM', 'geometry']]

In [8]:
data_gid

,POL_NM,geometry
0,노송지구대,"MULTIPOLYGON (((127.01079 37.31908, 127.01047 ..."
1,노송지구대,"MULTIPOLYGON (((126.98697 37.31147, 126.98705 ..."
2,노송지구대,"MULTIPOLYGON (((126.98105 37.32685, 126.98160 ..."
3,율천파출소,"MULTIPOLYGON (((126.97444 37.30857, 126.97466 ..."
4,노송지구대,"MULTIPOLYGON (((126.98169 37.30624, 126.98170 ..."
5,화서문지구대,"MULTIPOLYGON (((127.02641 37.29733, 127.02597 ..."
6,장안문지구대,"MULTIPOLYGON (((127.01135 37.31892, 127.01146 ..."
7,장안문지구대,"MULTIPOLYGON (((127.02290 37.30862, 127.02336 ..."
8,창룡문지구대,"MULTIPOLYGON (((127.03274 37.30082, 127.03258 ..."
9,장안문지구대,"MULTIPOLYGON (((127.03171 37.33646, 127.03162 ..."


In [9]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [20]:
data3 = pd.read_csv("data_final_MinMaxScaler.csv")
data3

,gid,geometry,경도,위도,여성인구비율,cctv개수,가로등개수,경찰서접근성,총점
0,다사51a19a,MULTIPOLYGON (((126.9473422052434 37.268619788...,126.950145,37.270886,0.236652,0.000000,0.078652,0.974799,1.132800
1,다사51a20a,MULTIPOLYGON (((126.9472763125004 37.277633381...,126.950080,37.279900,0.228014,0.000000,0.033708,0.905186,1.099493
2,다사59a21a,MULTIPOLYGON (((127.0374588078356 37.287034030...,127.040265,37.289298,1.000000,0.000000,0.082397,0.101242,1.018845
3,다사58b23b,MULTIPOLYGON (((127.0316785405179 37.309546037...,127.034486,37.311811,0.235469,0.000000,0.071161,0.806428,0.970736
4,다사62a21b,MULTIPOLYGON (((127.0712768150881 37.291668310...,127.074085,37.293932,0.227001,0.000000,0.123596,0.694449,0.797854
...,...,...,...,...,...,...,...,...,...
327,다사62a16b,MULTIPOLYGON (((127.0715322692303 37.246599477...,127.074338,37.248863,0.233027,0.666667,0.333333,0.092915,-0.674059
328,다사55b18b,MULTIPOLYGON (((126.9981245359784 37.264338996...,127.000929,37.266604,0.140993,0.666667,0.277154,0.060336,-0.742491
329,다사53a22a,MULTIPOLYGON (((126.9697090706761 37.295763777...,126.972514,37.298030,0.173139,0.500000,0.426966,0.000000,-0.753827
330,다사59b18b,MULTIPOLYGON (((127.0432355639738 37.264521633...,127.046042,37.266786,0.258794,1.000000,0.123596,0.091746,-0.773056


In [21]:
data3 = data3[["gid","위도","경도"]]

In [22]:
data3

,gid,위도,경도
0,다사51a19a,37.270886,126.950145
1,다사51a20a,37.279900,126.950080
2,다사59a21a,37.289298,127.040265
3,다사58b23b,37.311811,127.034486
4,다사62a21b,37.293932,127.074085
...,...,...,...
327,다사62a16b,37.248863,127.074338
328,다사55b18b,37.266604,127.000929
329,다사53a22a,37.298030,126.972514
330,다사59b18b,37.266786,127.046042


In [23]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-23-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-23-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-23-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [24]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['POL_NM'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|███████████████████████████████████████████████████████████████████████████████| 332/332 [00:00<00:00, 483.21it/s]


In [25]:
gid_list

['호매실지구대',
 '호매실지구대',
 '동부파출소',
 '광교지구대',
 '산남지구대',
 '노송지구대',
 '장안문지구대',
 '서호지구대',
 '산남지구대',
 '광교지구대',
 '고색파출소',
 '장안문지구대',
 '호매실지구대',
 '산남지구대',
 '고색파출소',
 '호매실지구대',
 '고색파출소',
 '매탄지구대',
 '산남지구대',
 '산남지구대',
 '산남지구대',
 '호매실지구대',
 '고색파출소',
 '장안문지구대',
 '장안문지구대',
 '광교지구대',
 '호매실지구대',
 '당수파출소',
 '장안문지구대',
 '태장파출소',
 '동부파출소',
 '고색파출소',
 '곡선지구대',
 '호매실지구대',
 '서호지구대',
 '행궁파출소',
 '광교지구대',
 '호매실지구대',
 '당수파출소',
 '고색파출소',
 '노송지구대',
 '장안문지구대',
 '당수파출소',
 '노송지구대',
 '매탄지구대',
 '매산지구대',
 '서호지구대',
 '영통지구대',
 '호매실지구대',
 '세류지구대',
 '서호지구대',
 '당수파출소',
 '광교지구대',
 '산남지구대',
 '노송지구대',
 '노송지구대',
 '호매실지구대',
 '호매실지구대',
 '산남지구대',
 '산남지구대',
 '매탄지구대',
 '노송지구대',
 '태장파출소',
 '서호지구대',
 '영통지구대',
 '동부파출소',
 '산남지구대',
 '율천파출소',
 '곡선지구대',
 '영통지구대',
 '영통지구대',
 '세류지구대',
 '당수파출소',
 '태장파출소',
 '인계파출소',
 '고색파출소',
 '장안문지구대',
 '광교지구대',
 '장안문지구대',
 '권선파출소',
 '산남지구대',
 '매탄지구대',
 '장안문지구대',
 '인계파출소',
 '노송지구대',
 '호매실지구대',
 '광교지구대',
 '세류지구대',
 '산남지구대',
 '호매실지구대',
 '고색파출소',
 '화서문지구대',
 '호매실지구대',
 '서호지구대',
 '산남지구대',
 '광교지구대',
 '고등파출소',
 '서호지구

In [26]:
data3['gid'] = gid_list


In [28]:
data3

,gid,위도,경도,sub_cnt,point
0,호매실지구대,37.270886,126.950145,1,POINT (126.95015 37.27089)
1,호매실지구대,37.279900,126.950080,1,POINT (126.95008 37.27990)
2,동부파출소,37.289298,127.040265,1,POINT (127.04027 37.28930)
3,광교지구대,37.311811,127.034486,1,POINT (127.03449 37.31181)
4,산남지구대,37.293932,127.074085,1,POINT (127.07408 37.29393)
...,...,...,...,...,...
327,영통지구대,37.248863,127.074338,1,POINT (127.07434 37.24886)
328,매산지구대,37.266604,127.000929,1,POINT (127.00093 37.26660)
329,율천파출소,37.298030,126.972514,1,POINT (126.97251 37.29803)
330,매탄지구대,37.266786,127.046042,1,POINT (127.04604 37.26679)


In [29]:
data3.to_csv("output/suwon_police_allocate.csv")

In [17]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [18]:
data3

,gid,TL_cnt
0,고등파출소,4
1,고색파출소,26
2,곡선지구대,5
3,광교지구대,23
4,권선파출소,17
5,노송지구대,32
6,당수파출소,12
7,동부파출소,10
8,매산지구대,3
9,매탄지구대,18
